In [1]:
import pandas as pd
import openai
import os
from tqdm import tqdm

from openai import OpenAI, OpenAIError


In [2]:
client = OpenAI()

In [3]:
completion = client.chat.completions.create(
  model="gpt-4-turbo-2024-04-09",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I help you today?


In [4]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo-2024-04-09",  # Using the specific model as indicated
            messages=messages,
            temperature=1,
            top_p=1
        )
        return response.choices[0].message.content  # Accessing the 'content' using dot notation
    except OpenAIError as e:
        print(f"OpenAI Error: {e} - There was an issue with the API call.")
        return None  # Handle OpenAI API specific errors
    except Exception as e:
        print(f"General Error: {e} - An unexpected error occurred.")
        return None  # Handle other unforeseen errors

# Example usage
prompt = "hello!"
output_sentence = get_completion(prompt)
print(output_sentence if output_sentence else "Prompt was skipped or an error occurred.")


# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-4-turbo-2024-04-09",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)

Hello! How can I assist you today?


In [5]:
def load_examples(file_path):
    with open(file_path, 'r') as file:
        examples = file.read().strip()
    return examples

In [6]:
def generate_data(csv_input, csv_output, examples_file):
    # Load the examples for few-shot learning
    examples = load_examples(examples_file)
    # Load the CSV file
    goldstandard = pd.read_csv(csv_input)


    # ------------------------------test
    # goldstandard = goldstandard.head(5) 

    
    # Initialize the output DataFrame
    generated_data = []

    # Process each row in the DataFrame
    for idx, row in tqdm(goldstandard.iterrows(), total=goldstandard.shape[0], desc="Generating Data"):
        prompt = f"{examples}\n\n" \
                 f"INSTRUCTION: Write a sentence with the semantic meaning the same as [{row['#CUE_COLUMN']}], " \
                 f"use the template [{row['FRAGMENT_COLUMN']}], the sentence should be euphemistic not explicit, " \
                 f"the sentence should be [{row['LABEL_COLUMN']}]." \
                 f"\n\n OUTPUT:"
        generated_sentence = get_completion(prompt)
        generated_data.append({
            '#CUE_COLUMN': row['#CUE_COLUMN'],
            'FRAGMENT_COLUMN': row['FRAGMENT_COLUMN'],
            'INSTANCE_COLUMN': generated_sentence,
            'LABEL_COLUMN': row['LABEL_COLUMN']
        })

    # Create a DataFrame from the generated data
    new_df = pd.DataFrame(generated_data)
    new_df.to_csv(output_csv, index=False)
    print("Data generation complete and saved to", csv_output)

In [7]:
input_csv = '../modified_goldstandard.csv'
# output_csv = 'gpt4_augmented_data_test.csv'
output_csv = 'gpt4_augmented_data.csv'
examples_file = 'gpt4_prompt.txt'

In [8]:
generate_data(input_csv, output_csv, examples_file)

Generating Data: 100%|██████████| 1797/1797 [37:57<00:00,  1.27s/it]

Data generation complete and saved to gpt4_augmented_data.csv
